In [1]:
path = './ntust-ir-2020_hw5_new/'
# path = '../HW2-BM25/ntust-ir-2020/'
import numpy as np
import numba as nb
import time
import math
from tqdm import tqdm
from math import log
from collections import Counter
from numba.typed import List
from scipy import spatial
# from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# data of doc and query
doc = []
query = []
# length of each doc
doc_len = []
# Name of doc and query
doc_list = []
query_list = []
# TF of doc and query
tf_doc = []
tf_q = []
# BM25 query vocabulary
bm25_vocabulary = set()
# Document IDF
doc_idf = dict()
# -----------------------------------------------------------------------------
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        tf_q.append(Counter(temp_query))
        for w in temp_query:
            bm25_vocabulary.add(w)
        query.append(temp_query)
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        for k in Counter(temp_doc).keys():
            if(k in doc_idf):
                doc_idf[k] += 1
            else: doc_idf[k] = 1
        tf_doc.append(Counter(temp_doc))
        doc.append(temp_doc)
        doc_len.append(len(temp_doc))
        f_temp.close()

100%|██████████| 30000/30000 [02:55<00:00, 171.09it/s]


In [3]:
# vsm vocabulary
vsm_voc = set()
bm25_vocabulary = list(bm25_vocabulary)
# Doc, query and vocabulary length
d_len = len(doc)
q_len = len(query)
bm_w_len = len(bm25_vocabulary)

In [5]:
# Build TF_key and TF_val
TF_doc_k = List()
TF_doc_v = List()
for d in tqdm(range(d_len)):
    TF_doc_k.append(List([k for k in tf_doc[d].keys()]))
    TF_doc_v.append(List([v for v in tf_doc[d].values()]))
TF_q_k = List()
TF_q_v = List()
for q in range(q_len):
    TF_q_k.append(List([k for k in tf_q[q].keys()]))
    TF_q_v.append(List([v for v in tf_q[q].values()]))

100%|██████████| 30000/30000 [00:34<00:00, 873.88it/s]


In [6]:
for w in tqdm(bm25_vocabulary):
    if(not(w in doc_idf)):
        doc_idf[w] = 0
        for d in range(d_len):
            if(w in tf_doc[d]):
                doc_idf[w] += 1

100%|██████████| 324/324 [00:00<00:00, 325108.73it/s]


In [7]:
# Build VSM vocabulary
for w in doc_idf.keys():
    if(doc_idf[w] > 5):
        vsm_voc.add(w)
vsm_voc = list(vsm_voc)
w_len = len(vsm_voc)
print(w_len)

31328


In [8]:
# BM25 IDF
@nb.njit
def bm_IDF(idf, tf_key, bm25_voc, dl, bm25l):
    for d in range(dl):
        for v in range(bm25l):
            if(bm25_voc[v] in tf_key[d]):
                idf[v] += 1 
    return idf

In [9]:
# BM25 IDF
bm_idf = np.zeros(bm_w_len)
time_idf = time.time()
bm_idf = bm_IDF(bm_idf, TF_doc_k, bm25_vocabulary, d_len, bm_w_len)
print('time: {:.5f} mins\n'.format((time.time() - time_idf) / 60))

time: 1.34534 mins



In [10]:
# B25 Model
@nb.njit
def BM25(sim, k1, b_, N_, avg, dl_list, dl, ql, tf_key, tf_val, tf_q_key, idf_, bm_voc):
    for q in range(ql):
        for d in range(dl):
            sum = 0.0
            for w in tf_q_key[q]:
                sum_tmp = 1.0
                if(w in tf_key[d]):
                    w_tf = tf_val[d][tf_key[d].index(w)]
                    ni_loc = bm_voc.index(w)
                    ni = idf_[ni_loc]
                    sum_tmp = (k1 + 1) * w_tf / (k1 * ((1 - b) + b * dl_list[d] / avg) + w_tf)
                    sum_tmp *= log((N_ - ni + 0.5) / (ni + 0.5))
                sum += sum_tmp
            sim[q][d] = sum
    return sim

In [11]:
# BM25 model variable
bm_sim = np.zeros([q_len, d_len], dtype=float)
K1 = 0.8
b = 0.7
N = d_len
avg_dl = np.average(doc_len)

In [12]:
time_bm = time.time()
bm_sim = BM25(bm_sim, K1, b, N, avg_dl, doc_len, d_len, q_len, TF_doc_k, TF_doc_v, TF_q_k, bm_idf, bm25_vocabulary)
print('time: {:.2f} mins\n'.format((time.time() - time_bm) / 60))

time: 2.15 mins



In [13]:
# Create output file
fp = open("bm.txt", "w")
fp.write("Query,RetrievedDocuments\n")
for i in range(q_len):
    fp.write('{},'.format(query_list[i]))
    for s in np.argsort(-bm_sim[i])[:5000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()

In [31]:
c = 0
for w in range(w_len):
    if(tf_doc[7][vsm_voc[w]] > 0):
        c += 1
print(c)
print(len(tf_doc[7].keys()))

114
117


In [32]:
# Build VSM Doc TF-IDF
vsm_doc_idf = np.zeros(w_len, dtype=float)
for w in tqdm(range(w_len)):
    vsm_doc_idf[w] = log(1 + N / (doc_idf[vsm_voc[w]] + 1)) # log(1 + N / ni+1)

100%|██████████| 31328/31328 [00:00<00:00, 661813.78it/s]


In [33]:
# TF
vsm_doc_tfidf = np.zeros([d_len, w_len], dtype=float)
for d in tqdm(range(d_len)):
    for w in range(w_len):
        vsm_doc_tfidf[d][w] = 1 + tf_doc[d][vsm_voc[w]]
print(vsm_doc_tfidf)

100%|██████████| 30000/30000 [11:25<00:00, 43.74it/s][[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]



In [38]:
c = 0
for d in range(d_len):
    if(tf_doc[d][vsm_voc[0]] > 0):
        c += 1
print(c)
print(doc_idf[vsm_voc[0]])

57
57


In [39]:
# TF-IDF
for d in tqdm(range(d_len)):
    vsm_doc_tfidf[d] = np.multiply(vsm_doc_tfidf[d], vsm_doc_idf)
print(vsm_doc_tfidf)

100%|██████████| 30000/30000 [00:01<00:00, 18668.47it/s][[6.25044112 6.84428286 6.84428286 ... 8.00670085 7.26512998 4.55124184]
 [6.25044112 6.84428286 6.84428286 ... 8.00670085 7.26512998 4.55124184]
 [6.25044112 6.84428286 6.84428286 ... 8.00670085 7.26512998 4.55124184]
 ...
 [6.25044112 6.84428286 6.84428286 ... 8.00670085 7.26512998 4.55124184]
 [6.25044112 6.84428286 6.84428286 ... 8.00670085 7.26512998 4.55124184]
 [6.25044112 6.84428286 6.84428286 ... 8.00670085 7.26512998 4.55124184]]



In [40]:
# Build VSM Query TF-IDF
vsm_query_tfidf = np.zeros([q_len, w_len], dtype=float)
# TF
for q in range(q_len):
    for w in range(w_len):
        vsm_query_tfidf[q][w] = 0.5 + 0.5 * tf_q[q][vsm_voc[w]]
# TF-IDF
for q in range(q_len):
    vsm_query_tfidf[q] = np.multiply(vsm_query_tfidf[q], vsm_doc_idf)

In [44]:
vsm_query_tfidf

array([[3.12522056, 3.42214143, 3.42214143, ..., 4.00335042, 3.63256499,
        2.27562092],
       [3.12522056, 3.42214143, 3.42214143, ..., 4.00335042, 3.63256499,
        2.27562092],
       [3.12522056, 3.42214143, 3.42214143, ..., 4.00335042, 3.63256499,
        2.27562092],
       ...,
       [3.12522056, 3.42214143, 3.42214143, ..., 4.00335042, 3.63256499,
        2.27562092],
       [3.12522056, 3.42214143, 3.42214143, ..., 4.00335042, 3.63256499,
        2.27562092],
       [3.12522056, 3.42214143, 3.42214143, ..., 4.00335042, 3.63256499,
        2.27562092]])

In [41]:
# Cos Sim
old_cos_sim = np.zeros([q_len, d_len], dtype=float)
for q in tqdm(range(q_len)):
    for d in range(d_len):
        old_cos_sim[q][d] = 1 - spatial.distance.cosine(vsm_query_tfidf[q], vsm_doc_tfidf[d])

100%|██████████| 150/150 [07:46<00:00,  3.11s/it]


In [43]:
old_cos_sim

array([[0.9993589 , 0.99419659, 0.99835489, ..., 0.99581334, 0.99950241,
        0.99314931],
       [0.99932916, 0.99416683, 0.99832676, ..., 0.99578356, 0.99947428,
        0.99312121],
       [0.99933224, 0.99417373, 0.99832985, ..., 0.99578188, 0.99947737,
        0.9931243 ],
       ...,
       [0.9993577 , 0.9941954 , 0.99835531, ..., 0.99580736, 0.99950283,
        0.99314973],
       [0.99934464, 0.99418232, 0.99834224, ..., 0.99579428, 0.99948976,
        0.99313668],
       [0.99934924, 0.99424972, 0.99835949, ..., 0.99579889, 0.99949437,
        0.99314128]])

In [42]:
# Create output file
fp = open("vsm.txt", "w")
fp.write("Query,RetrievedDocuments\n")
for i in range(q_len):
    fp.write('{},'.format(query_list[i]))
    for s in np.argsort(-old_cos_sim[i])[:5000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()

In [25]:
# Get relevance document
Rele_len = int(3)
Rq = np.zeros([q_len, Rele_len], dtype=np.int64)
for q in tqdm(range(q_len)):
    arg = np.argsort(-bm_sim[q])[:Rele_len]
    for r in range(Rele_len):
        Rq[q][r] = arg[r]

100%|██████████| 150/150 [00:00<00:00, 1262.74it/s]


In [26]:
# New Query
def N_query(new_q, old_q, rel_doc, rel_len, doc_tfidf, ql, wl, alpha_, beta_, iter_):
    for i in tqdm(range(iter_)):
        for q in range(ql):
                s1 = old_q[q] * alpha_
                s2 = beta_ / rel_len
                for d in range(rel_len):
                    s1 = np.vstack((s1, (doc_tfidf[rel_doc[q][d]] * s2)))
                new_q[q] = np.sum(s1, axis=0)
        old_q = np.copy(new_q)
    return new_q

In [27]:
# new query model
alpha = 1
beta = 0.75
iteration = 5
new_query = np.zeros([q_len, w_len])
new_query = N_query(new_query, vsm_query_tfidf, Rq, Rele_len, vsm_doc_tfidf, q_len, w_len, alpha, beta, iteration)

100%|██████████| 20/20 [00:07<00:00,  2.72it/s]


In [ ]:
new_query

In [33]:
# Cos Sim
cos_sim = np.zeros([q_len, d_len], dtype=float)
for q in tqdm(range(q_len)):
    for d in range(d_len):
        cos_sim[q][d] = 1 - spatial.distance.cosine(new_query[q], vsm_doc_tfidf[d])

100%|██████████| 150/150 [12:13<00:00,  4.89s/it]


In [34]:
cos_sim

array([[0.9994894 , 0.99565052, 0.99867143, ..., 0.99665996, 0.99959643,
        0.99434325],
       [0.99946428, 0.99562538, 0.99864689, ..., 0.99663391, 0.9995719 ,
        0.99431874],
       [0.99946698, 0.99563081, 0.9986496 , ..., 0.99663301, 0.99957461,
        0.99432145],
       ...,
       [0.99948873, 0.99564985, 0.99867135, ..., 0.99665477, 0.99959635,
        0.99434317],
       [0.99947732, 0.99563844, 0.99865994, ..., 0.99664336, 0.99958495,
        0.99433178],
       [0.99948157, 0.99568883, 0.99867347, ..., 0.99664761, 0.99958919,
        0.99433602]])

In [35]:
# Create output file
fp = open("roicchio_{}_{}_iter_{}.txt".format(alpha, beta, iteration), "w")
fp.write("Query,RetrievedDocuments\n")
for i in range(q_len):
    fp.write('{},'.format(query_list[i]))
    for s in np.argsort(-cos_sim[i])[:5000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()